In [ ]:
from datetime import date
from tqdm import tqdm
import shutil
import os

from thumbnails import ThumbnailDef, Thumbnailer, ThumbnailParams
from db_connect import DbConnector
from photo_export import PhotoExporter
from path_management import PathManager
from bucket_connect import BucketConnector


In [ ]:
from_date = date(2024, 7, 10)
to_date = date(2024, 7, 10)
# overwrite files in bucket
overwrite = True

In [ ]:
small = ThumbnailDef(120, 'small')
medium = ThumbnailDef(400, 'medium')
large = ThumbnailDef(1600, 'large')

sizes = [small, large, medium] 
thumbnailer = Thumbnailer(sizes)

In [ ]:
paths = PathManager(thumbnailer.name_modifiers)
test_connector = DbConnector('./test.postgres_config', paths)
prod_connector = DbConnector('./prod.postgres_config', paths)
exporter = PhotoExporter(paths, hours_before=3, hours_after=2)

games = prod_connector.get_games(from_date, to_date)

In [ ]:
for game in tqdm(games):
    exporter.export_photos_for_game(game)

At this point, review the photos in the "preview" directories and remove any that shouldn't be uploaded. Removing it from the preview directory is sufficient.

In [ ]:
to_thumbnail: list[ThumbnailParams] = []
for game in games:
    to_thumbnail.extend(exporter.get_photos_to_thumbnail(game))

In [ ]:
thumbnailer.thumbnail_many(to_thumbnail)

In [ ]:
bucket = BucketConnector('./prod.bucket_config', paths)

In [ ]:
for game in games:
    to_upload = exporter.get_photos_to_upload(game)
    print(f'Uploading/Validating {len(to_upload)} photos for {game.Name}')
    upload_params = []
    for photo in to_upload:
        root_path = paths.temp_dir(game, photo)
        upload_params.extend(bucket.get_upload_params(root_path, photo, override=overwrite))
    bucket.upload_many_files(upload_params, pool_size=25)
    prod_connector.import_resources(game, to_upload)

In [ ]:
prod_connector.remove_duplicate_thumbnails() 
# does not remove files from the bucket

In [ ]:
for game in games:
    root_path = paths.base_dir(game)
    if os.path.exists(root_path):
        shutil.rmtree(root_path)